# Festival Playlists

In [4]:
import os
import numpy as np
import pandas as pd
import requests
import json
import spotipy
from IPython.display import display

1. Use the Songkick API to get all the bands playing the festival
2. Use the Setlist.FM API to get the setlists
3. Use the Spotify API to create the playlists and add all the songs

### Set API credentials

In [5]:
setlistfm_api_key = os.getenv('SETLISTFM_API_KEY')
spotify_client_id = os.getenv('SPOTIFY_CLIENT_ID')
spotify_client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')

### Setlist FM

#### Plan of action
1. Given a lineup (list of band names), get their Musicbrainz identifiers (`mbid`) via `https://api.setlist.fm/rest/1.0/search/artists`
2. Retrieve the setlists for each artist using their `mbid` via `https://api.setlist.fm/rest/1.0/artist/{artist_mbid}/setlists
`

In [6]:
lineup = pd.read_csv(
    '/Users/adrialuz/Desktop/weekender.txt', header=None, names=['band'], encoding="ISO-8859-1"
)['band'].values

In [7]:
len(lineup)

26

In [8]:
lineup

array(['Fresquito y mango', 'Mainline magic circus', 'La zowi',
       'Chica gang', 'Marie davidson', 'Viagra boys', 'Alewya',
       'Kokoshca', 'Paco moreno', 'The thurston moore band', 'Soto asa',
       'Kings of convenience', 'Casero', 'Mogwai', 'Pa salieu',
       'Los planetas', 'Maria arnal i marcel bages', 'Benny sings',
       'Hinds', 'Danny L Harle', 'Unai muguruza', 'Lagarde',
       'Marina herlop', 'Renaldo i Clara', 'Wind atlas',
       'El tablao flamenco del capullo de jerez'], dtype=object)

In [9]:
artists_url = 'https://api.setlist.fm/rest/1.0/search/artists'

In [10]:
lineup_mbids = []
not_found = []

for name in lineup:
    req = requests.get(artists_url,
                       headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                       params={'artistName': name, 'p': 1, 'sort': 'relevance'}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(artists_url,
                           headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                           params={'artistName': name, 'p': 1, 'sort': 'relevance'}
        )
        i += 1
    
    if req.ok:
        artist_response = req.json()['artist']
        num_artists = len(artist_response)
        if num_artists > 1:
            for i in range(num_artists):
                if artist_response[i]['name'].lower() == name.lower():
                    mbid = artist_response[i]['mbid']
                    lineup_mbids.append({'name': name, 'mbid': mbid})
                    break
        elif num_artists == 1:
            mbid = artist_response[0]['mbid']
            lineup_mbids.append({'name': name, 'mbid': mbid})
        elif num_artists == 0:
            print(f'No results I think for {name}')
        else:
            print(f'WTF {name}?')
                    
    else:
        print(f'Couldn\'t find {name}')
        not_found.append(name)

Couldn't find Fresquito y mango
Couldn't find Mainline magic circus
Couldn't find Chica gang
Couldn't find Paco moreno
Couldn't find The thurston moore band
Couldn't find Maria arnal i marcel bages
Couldn't find Unai muguruza
Couldn't find Renaldo i Clara
Couldn't find El tablao flamenco del capullo de jerez


In [11]:
lineup_mbids

[{'name': 'La zowi', 'mbid': '5d704b62-ecb2-4129-93c5-6e40f9189fa8'},
 {'name': 'Marie davidson', 'mbid': '5e864744-89b6-4da8-b463-3ed601c22d8c'},
 {'name': 'Viagra boys', 'mbid': '9df0c0ec-619e-447d-a155-23e19d9c84ce'},
 {'name': 'Alewya', 'mbid': '49b5386b-a606-41b1-90e8-d9e7983b6f1b'},
 {'name': 'Kokoshca', 'mbid': '82eed032-a79f-42e1-b08b-2afd6db0ad40'},
 {'name': 'Soto asa', 'mbid': '34a034a5-cc93-4c92-9c01-c0cfd27f823f'},
 {'name': 'Kings of convenience',
  'mbid': 'cf0f4547-ffbd-4011-98ad-0bec9ba022db'},
 {'name': 'Casero', 'mbid': '9e712d3d-b6e9-4f31-ba02-2c844d9adf72'},
 {'name': 'Mogwai', 'mbid': 'd700b3f5-45af-4d02-95ed-57d301bda93e'},
 {'name': 'Pa salieu', 'mbid': '03100b87-7d27-4d56-b049-2007599656cc'},
 {'name': 'Los planetas', 'mbid': '6f28b158-4d1a-4f31-9339-7f381e3c4ee1'},
 {'name': 'Benny sings', 'mbid': 'cdaf7932-ab05-4f71-8721-104da6e6b8d0'},
 {'name': 'Hinds', 'mbid': '615ba4a9-e5d2-45dc-bf75-a1b444a33fc9'},
 {'name': 'Danny L Harle', 'mbid': '1fd94d42-5e6a-46f2-8

In [12]:
not_found

['Fresquito y mango',
 'Mainline magic circus',
 'Chica gang',
 'Paco moreno',
 'The thurston moore band',
 'Maria arnal i marcel bages',
 'Unai muguruza',
 'Renaldo i Clara',
 'El tablao flamenco del capullo de jerez']

In [13]:
artist_setlist = []

for a in lineup_mbids:
    songs_played = []
    mbid = a['mbid']
    setlists_url = f'https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists'
    
    req = requests.get(setlists_url,
             headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
             params={'p': 1}
    )
    
    i = 0
    while (not req.ok) & (i <= 5):
        req = requests.get(setlists_url,
                 headers={'x-api-key': setlistfm_api_key, 'Accept': 'application/json'},
                 params={'p': 1}
        )
        i += 1
            
    if req.ok:
    
        setlist_response = req.json()['setlist']
        num_setlists = len(setlist_response)

        for i in range(num_setlists):
            setlist = setlist_response[i]['sets']['set']
            num_sections = len(setlist)
            total_songs = []
            for p in range(num_sections):
                total_songs += setlist[p]['song']
            num_songs = len(total_songs)

            for i in range(num_songs):
                song = total_songs[i]
                song_title = song['name']
                # if the song is a cover add the original artist to the song title
                if 'cover' in song:
                    song_title += ' {}'.format(song['cover']['name'])
                songs_played.append(song_title)
                                           
        most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)

        artist_setlist.append({
            'artist': a['name'],
            'setlist': most_played_songs
        })
    else:
        not_found.append(a['name'])

/var/folders/5g/z2_s3_7d75qgyjp72n_8q0p40000gq/T/ipykernel_81509/1942038417.py:42: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  most_played_songs = list(pd.Series(songs_played).value_counts().head(15).index)


In [14]:
not_found

['Fresquito y mango',
 'Mainline magic circus',
 'Chica gang',
 'Paco moreno',
 'The thurston moore band',
 'Maria arnal i marcel bages',
 'Unai muguruza',
 'Renaldo i Clara',
 'El tablao flamenco del capullo de jerez']

In [15]:
artist_setlist

[{'artist': 'La zowi',
  'setlist': ['No Lo Ves',
   'Bitch Mode',
   'Trust No Bitch',
   'Putas (Remix)',
   'Bitch Te Quemas',
   'Oscuro',
   'Obra de arte',
   'Tu o Yo',
   'Thotie of the Year',
   '(Unknown)',
   'Si Te Pillo (ft la Albany)',
   'Pussy Poppin (ft la Goony Chonga)',
   'Tu o Yo (con interludio)*',
   'Filet Mignon',
   'Sugar mami']},
 {'artist': 'Marie davidson',
  'setlist': ['The Psychologist',
   'Workaholic Paranoid Bitch',
   'Lara',
   'Work It',
   'Excès de vitesse',
   'Naive to the Bone',
   'So Right',
   'Adieu Au Dancefloor',
   'Chasing The Light',
   'Planète Ego']},
 {'artist': 'Viagra boys',
  'setlist': ['Slow Learner',
   'Just Like You',
   'Down in the Basement',
   'Sports',
   'Shrimp Shack',
   'Research Chemicals',
   'Worms',
   'Frogstrap',
   'Amphetanarchy',
   'Call of the Wild',
   'Femtaktam',
   'Liquids',
   'Frijazz',
   'Up All Night',
   "I Don't Remember That"]},
 {'artist': 'Alewya', 'setlist': []},
 {'artist': 'Kokoshca',


In [16]:
setlist_lengths = []
short_or_empty_setlist = []

for i in range(len(artist_setlist)):
    n_songs = len(artist_setlist[i]['setlist'])
    setlist_lengths.append({
        'artist': artist_setlist[i]['artist'],
        'n_songs': n_songs
    })
    
    if n_songs < 5:
        short_or_empty_setlist.append(artist_setlist[i]['artist'])

In [17]:
len(short_or_empty_setlist)

4

In [18]:
short_or_empty_setlist

['Alewya', 'Soto asa', 'Pa salieu', 'Marina herlop']

### Spotify

In [34]:
username = 'adrialuz'
scope = 'playlist-modify-public'

token = spotipy.util.prompt_for_user_token(username, scope, redirect_uri='http://localhost:9090')

sp = spotipy.Spotify(auth=token)

sp.trace = False

In [20]:
sp.search('artist:Dua Lipa', limit=1, type='artist', market='GB')['artists']['items'][0]['id']

'6M2wZ9GZgrQXHCFfjv46we'

In [21]:
sp.search(
        f'artist:Khaled', limit=2, type='artist', market='GB'
    )['artists']['items']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/0QHgL1lAIqAw0HtD7YldmP'},
  'followers': {'href': None, 'total': 6468234},
  'genres': ['hip hop',
   'miami hip hop',
   'pop',
   'pop rap',
   'rap',
   'southern hip hop',
   'trap'],
  'href': 'https://api.spotify.com/v1/artists/0QHgL1lAIqAw0HtD7YldmP',
  'id': '0QHgL1lAIqAw0HtD7YldmP',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/ab6761610000e5ebbf921114b7f19be97ce29647',
    'width': 640},
   {'height': 320,
    'url': 'https://i.scdn.co/image/ab67616100005174bf921114b7f19be97ce29647',
    'width': 320},
   {'height': 160,
    'url': 'https://i.scdn.co/image/ab6761610000f178bf921114b7f19be97ce29647',
    'width': 160}],
  'name': 'DJ Khaled',
  'popularity': 80,
  'type': 'artist',
  'uri': 'spotify:artist:0QHgL1lAIqAw0HtD7YldmP'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/28ztjHIXceRRntmTUfnmUX'},
  'followers': {'href': None, 'total': 1318648},
  'genres': ['classic arab p

In [22]:
spotify_ids = []
for a in short_or_empty_setlist:
    search_result = sp.search(
        f'artist:{a}', limit=5, type='artist', market='GB'
    )['artists']['items']
    
    if search_result:
        for i in range(len(search_result)):
            name = search_result[i]['name']
            if name.lower() == a.lower():
                artist_id = search_result[i]['id']
                spotify_ids.append(artist_id)
                break
            else:
                pass
    else:
        print(f'Couldn\'t find {a} on Spotify.')

In [23]:
spotify_ids

['0wcjJjpvnHb5vK4iwKfxPm',
 '6gyhsZUy2fzzWZBRVPmOw4',
 '290nCNEce1y6rfoJiO2rK7',
 '4ZG4S6skttdVhHz08AjMhR']

In [24]:
sp.artist('59xdAObFYuaKO2phzzz07H')['name']

'Special Request'

In [25]:
popular_songs = []

for artist_id in spotify_ids:
    search_results = sp.artist_top_tracks(artist_id, country='GB')['tracks']

    top_songs = []
    if search_results:
        for i in range(len(search_results)):
            song_name = search_results[i]['name']
            top_songs.append(song_name)
        popular_songs.append({
            'artist': sp.artist(artist_id)['name'],
            'setlist': top_songs
        })
    else:
        print(artist_id, sp.artist(artist_id)['name'])

In [26]:
popular_songs

[{'artist': 'Alewya',
  'setlist': ['Sweating',
   'Spirit_X',
   'where’s my lighter',
   'Play',
   'The Code',
   'Sweating - Breakage Remix',
   'Jagna',
   'Sweating (Honey Dijon Remix) - Edit',
   'Sweating - Honey Dijon Remix']},
 {'artist': 'Soto Asa',
  'setlist': ['Smartphone',
   'Si Tu Kiere',
   'Me Gustaron Tus Nai',
   'Klub',
   'Kiyaera',
   'Christian Dior',
   'Hadu',
   'Yo Te Buské',
   'Otra Realidad',
   'Dolce']},
 {'artist': 'Pa Salieu',
  'setlist': ['Glidin’ (feat. slowthai)',
   'Energy (feat. Mahalia)',
   'Lit',
   'Hit The Block (with Pa Salieu)',
   'Like Tu Danz',
   'Informa (feat. M1llionz)',
   'Style & Fashion (feat. Obongjayar)',
   'My Family (feat. BackRoad Gee)',
   'Shining (feat. Tay Iwar & Zlatan)',
   'Frontline']},
 {'artist': 'Marina Herlop',
  'setlist': ['miu',
   'Doiloi',
   'Heks',
   'Odessa',
   'Hval',
   'Snu',
   'Bus II',
   'Bange',
   'Svane',
   'Goor']}]

Get the URI codes for each track

In [27]:
uris = []
missing_songs = []

for a in (artist_setlist + popular_songs):
    artist = a['artist']
    setlist = a['setlist']
    for s in setlist:
        s = s.replace(',', '').replace('\'', '').replace('"', '').replace('.', '').replace(
            '?', '').replace(')', '').replace('(', '').replace('/', '').replace(
            '\\', '').replace('&', '').replace('-', '')
        items = sp.search(q=f'artist:{artist} track:{s}', limit=1)['tracks']['items']
        if items:
            uri = items[0]['id']
            uris.append(uri)
        else:
            items = sp.search(q=f'track:{s}', limit=1)['tracks']['items']
            if items:
                if items != [None]:
                    uri = items[0]['id']
                    uris.append(uri)
            else:
                missing_songs.append({
                    'artist': artist,
                    'song': s
                })

In [28]:
len(uris)

194

In [29]:
len(missing_songs)

11

In [30]:
missing_songs

[{'artist': 'La zowi', 'song': 'Si Te Pillo ft la Albany'},
 {'artist': 'La zowi', 'song': 'Pussy Poppin ft la Goony Chonga'},
 {'artist': 'La zowi', 'song': 'Tu o Yo con interludio*'},
 {'artist': 'Viagra boys', 'song': 'Femtaktam'},
 {'artist': 'Danny L Harle',
  'song': 'Panda Bear  Come To Your Senses Danny L Harle Remix'},
 {'artist': 'Danny L Harle',
  'song': 'Call Me Maybe Audio Paradyne Bootleg Carly Rae Jepsen'},
 {'artist': 'Danny L Harle',
  'song': 'Blue Angel feat Clairo DJ Danny Harlecore Remix'},
 {'artist': 'Danny L Harle', 'song': 'SOPHIE  Ponyboy Danny L Harle Edit'},
 {'artist': 'Danny L Harle',
  'song': 'Dj Paul Elstak  Rainbow In The Sky Danny L Harle Edit'},
 {'artist': 'Danny L Harle',
  'song': 'Ariana Grande  No Tears Left To Cry Danny L Harle Edit'},
 {'artist': 'Danny L Harle', 'song': 'Take My Heart Away feat Eyelar'}]

In [31]:
divisor = int(np.floor(len(uris) / np.ceil(len(uris) / 100)))
times = int(np.floor(len(uris) / divisor))

In [37]:
for i in range(times):
    subset = uris[divisor*i:divisor*(i+1)]
    sp.user_playlist_add_tracks(username, playlist_id='2nUkznVEo8EgQXw0UucbpS',
                                tracks=subset)